In [ ]:
# This script converts the trained "model.h5" into OpenVINO MO.
# https://docs.openvino.ai/2022.3/openvino_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_TensorFlow.html#keras-h5 
import time
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from IPython.display import Markdown
from openvino.runtime import Core

In [ ]:
model_h5_dir = Path("./files/A_20240630")
model_h5_path = model_h5_dir / "model.h5"

model_path = Path("./files/A_20240630/saved_model.pb")

ir_path = Path("model/A_20240530/NanoNet-fp32.xml")

In [ ]:
from metrics import iou, dice_coef, dice_loss, bce_dice_loss
from tensorflow.keras.utils import CustomObjectScope

# Load the model
with CustomObjectScope({
        'iou':iou,
        'dice_coef':dice_coef,
        'dice_loss':dice_loss,
        'bce_dice_loss': bce_dice_loss
    }):
    try:
        model = tf.keras.models.load_model(model_h5_path)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Failed to load model: {e}")
        

In [ ]:
saved_model_path =  model_h5_dir /'model'
tf.saved_model.save(model,saved_model_path)


In [ ]:
import openvino as ov
ov_model = ov.convert_model(saved_model_path, input=[1, 256, 256, 3]) 

In [ ]:
ov.save_model(ov_model, saved_model_path /'nanonet-fp32-shape-1-256-256-3-model.xml')